In [ ]:
import pandas as pd
import numpy as np
#surpress divide warnings
np.errstate(invalid='ignore', divide='ignore')
import matplotlib.pyplot as plt
import pickle
from src.data_tools.get_data import get_data

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.plotting_tools.draw_stack_plot_hists import draw_bckground, draw_signals, draw_data, draw_stackplot
from src.plotting_tools.SysHist import SysHist
from src.plotting_tools.Bins import Bins, bins
from src.plotting_tools.utils import ratio_plot_template
from src.data_tools.StackPlotter import get_stack_plotter
from src.plotting_tools.latexAssets import mll
from src.assets.output_dir import output_dir
output_dir

In [ ]:
split_bins = bins

In [ ]:
from src.general.array_utils import moving_average, moving_sum, super_sample, super_sample_function, moving_avg_func, unp_array_to_nom_std
from src.plotting_tools.cms_format import cms_style, cms_format_fig

In [ ]:
from scipy.optimize import curve_fit
from src.general.functions import power_func, power_law, make_bpoly, linear, parabola, make_bpoly_exp

In [ ]:
from scipy.optimize import curve_fit
from src.general.functions import make_bpoly, lognorm, log_norm_np, log_norm_unp
from src.plotting_tools.SysHist import SysHist
import uncertainties
import uncertainties.unumpy as unp

In [ ]:
cms_style()

In [ ]:
#output_dir = 'assets_feb_23'
outdir = '{}/abcd'.format(output_dir)
era = '2016'
ismc=0
isdata = ismc==0

In [ ]:
sp = get_stack_plotter(output_dir, era, bins='none')

In [ ]:
def compare_hists(fhist, dhist, ismc_pull = False, n=10, ndof=0, log=1, flabel="", dlabel="", 
                  dhist_isdata=0, fhist_is_data=0, ratio=False,  isabcd=0, **kwargs):
    fcolor = 'green' if isabcd else 'blue'
    
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    #top plots
    fhist.draw(ax, label=flabel, zorder=3,color=fcolor)
    if dhist_isdata:
        ax.errorbar(dhist.calc_bin_centers(), dhist.nominal, yerr=dhist.std, color='black', label=dlabel, ls='', marker='o', zorder=3)
    else:
        dhist.draw(ax, label=dlabel, zorder=1, errorbar=False, draw_sys=0, color='red', alpha=0)#, sys_label='Background Systematics')
    x = fhist.calc_bin_centers()
    
    #calc pull
    nom1_avg = moving_sum(fhist.nominal, n = n)
    nom2_avg = moving_sum(dhist.nominal, n = n)
    var1_avg = moving_sum(fhist.std**2, n = n)
    var2_avg = moving_sum(dhist.std**2, n = n)
    x_avg = moving_average(x, n=n)
    if ismc_pull:
        pull = (nom1_avg-nom2_avg)/(var2_avg+var1_avg)**.5
    else:
        pull = (nom1_avg-nom2_avg)/nom1_avg**.5
    pullsquare = pull**2
    
    if ratio:
        rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
        rax.plot(x, np.full(len(x), 0) , color='black')
        rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 

        if isabcd:
            fhist.calc_ratio(fhist.nominal).draw(rax, zorder=1, color=fcolor)
        else:
            fhist.calc_ratio(fhist.nominal).draw(rax, zorder=1)
        if dhist_isdata:
            rdhist = dhist.calc_ratio(fhist.nominal)
            rax.errorbar(rdhist.calc_bin_centers(), rdhist.nominal, yerr=rdhist.std, color='black', label=dlabel, ls='', marker='o', zorder=3)
        else:
            dhist.calc_ratio(fhist.nominal).draw(rax, label=dlabel, zorder=0, errorbar=False, color='red', alpha=0.5)
            #dhist.calc_ratio(fhist.nominal).draw(rax, zorder=0, errorbar=False)
    else:
        rax.plot(x, np.full(len(x), 1) , color='black', linestyle=':')
        rax.plot(x, np.full(len(x), 0) , color='black')
        rax.plot(x, np.full(len(x), -1) , color='black', linestyle=':')
        rax.plot(x_avg, pull)
    
    #format ax
    if isdata:
        cms_format_fig(era, ax, "\emph{Preliminary}")
    else:
        cms_format_fig(era, ax, "\emph{Simulation}")
    if log:
        ax.set_yscale('Log')
            
    ax.set_ylabel('Counts per GeV')
    
    #format rax
    rax.set_ylim(0,2)
    if ratio:
        if dhist_isdata:
            rax.set_ylabel('Obs./Fit')
            if isabcd: rax.set_ylabel('Obs./ABCD')
        else:
            if isabcd: rax.set_ylabel('MC/ABCD')
            else: rax.set_ylabel('MC/Fit')
    else:
        rax.set_ylim(-5,5)
        rax.set_ylabel('Pull')
        
    rax.set_xlabel('{} [GeV]'.format(mll))
    return  {'chi2': (pullsquare).sum()/(fhist.calc_nBins()-ndof),
             'fig': fig,
             'ax': ax,
             'rax': rax}
    

In [ ]:
def fit_hist(func, hist, n=10, comp_hist_n =10, do_super_sample=1, ismc=False,  do_unc=1, color='red', 
             flabel="", dlabel="", dhist_isdata=0, fhist_is_data=0, **kwargs):
    x = np.array(hist.calc_bin_centers())
    if not isdata:
        popt, pcov = curve_fit(func, x, hist.nominal, 
                       **kwargs,
                       sigma=hist.std, maxfev = int(1e6))  
    else:
        #std is not optimal for data: zero and low count bins will be subotimal error estimates
        popt, pcov = curve_fit(func, x, hist.nominal, 
               **kwargs, maxfev = int(1e6))  
    if do_unc:
        #create fit values with uncertainties
        popt_unc = uncertainties.correlated_values(popt, pcov)
        #make_fit_hist
        y = log_norm_unp(x, *popt_unc)
        y_nom, y_std = unp_array_to_nom_std(y)
    else: 
        y_nom = func(x, *popt)
        varper = ((hist.nominal-y_nom)**2/y_nom).mean()
        y_std = (varper*y_nom)**.5
        y_std = y_nom**.5
    fit_hist = SysHist(
            y_nom,
            x*0, x*0, 
            y_std,
            np.array(hist.bin_edges)
        ).normalize().calc_ratio(1/hist.calc_sum())

    compare_dict = compare_hists(fit_hist, hist, ismc=ismc, n=comp_hist_n, color = color, ndof=5, flabel=flabel, dlabel=dlabel, dhist_isdata=dhist_isdata)
    
    return {**compare_dict, 
            "popt" : np.array(popt),
            "pcov": np.array(pcov),
            "fit_hist": fit_hist
           }

In [ ]:
bottom_value = 120
top_value=405
feature='DiLepMass'
sp.x_range = (bottom_value, top_value)

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
comp_hist_n = 1
sp.rebin = 0
fit_dict = {}
for reg in ['CR14','CR24', 'CR10','CR20', 'CR13', 'CR23']:
    #data for fitting, data or MC?
    if isdata:
        _plot_dict = sp.make_data_hist(feature, reg)
    else:
        _plot_dict = sp.combine_back(feature, reg)
    #reduce the range for fitting
    hist = _plot_dict.reduce_range(bottom=bottom_value, top=top_value)
    if ismc: hist.std += gaussian_filter(hist.std,2)
    #hist.up *=0
    #hist.down *=0
    total_events = hist.nominal.sum()
    #fit the hist
    flabel='Observed Fit' if isdata else 'MC Fit'
    dlabel = 'Observed' if isdata else None
    # fit and pull plot
    
    
    curve_fit_chi2 = fit_hist(log_norm_np, hist, comp_hist_n=comp_hist_n, do_super_sample=0, 
                              ismc=ismc, p0=[total_events*10, .8, 80, 70], 
                              bounds = ([0, .2, 50, 50], [total_events*100, 1, 100, 100]),
                              do_unc=1, flabel=flabel, dlabel=dlabel, dhist_isdata=isdata,fhist_is_data=isdata,
                             )
    
    
    sp.draw_background(curve_fit_chi2['ax'], feature, reg, sys_label='Stat. + Sys.', errorbar=False)
    print(reg)
    fit_dict[reg] = curve_fit_chi2
    print(reg, curve_fit_chi2['chi2'],  repr(curve_fit_chi2['popt']))
    # reorder legend
    handles, labels = curve_fit_chi2['ax'].get_legend_handles_labels()
    order = [0,1,2,3,4,5,7,6]
    if ismc: order = np.linspace(0,len(handles)-1, len(handles), dtype=int)
    curve_fit_chi2['ax'].legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                                ncol=2)
    curve_fit_chi2['ax'].set_ylim(bottom=1e0, top=1e4)
    curve_fit_chi2['fig'].savefig('{}/fit_data_pull_era{}_ismc{}_reg{}_bottom{}_lognorm.pdf'.format(outdir,era,ismc,reg,bottom_value))
    plt.show()
    #200gev pull
    curve_fit_chi2['ax'].set_xlim(left=bottom_value, right=200)
    curve_fit_chi2['rax'].set_xlim(left=bottom_value, right=200)
    curve_fit_chi2['fig'].savefig('{}/fit_data_pull_era{}_ismc{}_reg{}_bottom{}_lognorm_200GeVMax.pdf'.format(outdir,era,ismc,reg,bottom_value))

    ### ratio plot
    #rebin to split binning for these plots
    sp.rebin = 0 #split_bins.bin_edges
    bhist = sp.combine_back(feature, reg).make_density_hist()
    bhist.nominal += 1e-10
    fit_hist_rebin = curve_fit_chi2['fit_hist'].reduce_range(bottom=bottom_value, top=top_value).make_density_hist()
    ratio_dict = compare_hists(fit_hist_rebin, bhist, n=comp_hist_n, 
                               color = 'red', ndof=5, flabel=flabel, ratio=True, dhist_isdata=0, fhist_is_data=isdata)
    ratio_dict['ax'].legend()
    sp.draw_background(ratio_dict['ax'], feature, reg, sys_label='Stat. + Sys.', errorbar=False)
    if isdata:
        data_plot = sp.make_data_hist(feature, reg).make_density_hist()
        ratio_dict['ax'].errorbar(data_plot.calc_bin_centers(), data_plot.nominal, yerr=data_plot.std, 
                    color='black', label='Observed', ls='', marker='o', zorder=2)
        
        ratio_dict['rax'].errorbar(data_plot.calc_bin_centers(), data_plot.nominal/bhist.nominal,
                                   yerr=data_plot.std/bhist.nominal, 
                    color='black', ls='', marker='o', zorder=.5)        
        
    handles, labels = ratio_dict['ax'].get_legend_handles_labels()
    order = [0,1,2,3,4,6,5,7]
    if ismc: order = np.linspace(0,len(handles)-1, len(handles), dtype=int)
    ratio_dict['ax'].legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                                ncol=2)

    ratio_dict['ax'].set_ylim(bottom=1e0, top=1e4)
    ratio_dict['fig'].savefig('{}/fit_mc_ratio_era{}_ismc{}_reg{}_bottom{}_lognorm.pdf'.format(outdir,era,ismc,reg,bottom_value))
    plt.show()
    #200gev pull
    ratio_dict['ax'].set_xlim(left=bottom_value, right=200)
    ratio_dict['rax'].set_xlim(left=bottom_value, right=200)
    ratio_dict['fig'].savefig('{}/fit_mc_ratio_era{}_ismc{}_reg{}_bottom{}_lognorm_200GeVMax.pdf'.format(outdir,era,ismc,reg,bottom_value))
    sp.rebin = 0
    




In [ ]:
#### abcd

In [ ]:
def make_fit_hist(template_hist, reg):
    tmp_xrange = sp.x_range
    sp.x_range = (-np.inf, np.inf)
    template_hist = sp.combine_back(feature, reg)
    x = np.array(template_hist.calc_bin_centers())
    popt, pcov = fit_dict[reg]['popt'], fit_dict[reg]['pcov']
    popt_unc = uncertainties.correlated_values(popt, pcov)
    y = log_norm_unp(x, *popt_unc)
    y_nom, y_std = unp_array_to_nom_std(y)
    fit_hist = SysHist(
            y_nom,
            x*0, x*0, 
            y_std,
            np.array(template_hist.bin_edges)
        )
    sp.x_range = tmp_xrange
    return fit_hist

In [ ]:
def make_abcd(nJets):
    from copy import deepcopy
    A = deepcopy(make_fit_hist(bhist, 'CR{}0'.format(nJets)))
    B = deepcopy(make_fit_hist(bhist, 'CR{}3'.format(nJets)))
    C = deepcopy(make_fit_hist(bhist, 'CR{}4'.format(nJets)))
    abcd = A.uncertainty_std_dev()*B.uncertainty_std_dev()/C.uncertainty_std_dev()
    print( A.uncertainty_std_dev().sum(), B.uncertainty_std_dev().sum(), C.uncertainty_std_dev().sum(), abcd.sum())
    abcd_nom, abcd_std = unp_array_to_nom_std(abcd)
    return SysHist(abcd_nom, abcd_nom*0,abcd_nom*0,abcd_std, A.bin_edges)

In [ ]:
#abcd plots
for nJets in [1,2]:
    reg = 'SR{}'.format(nJets)
    #make abcd hist
    abcd_hist = make_abcd(nJets)
    abcd_hist = abcd_hist.rebin(split_bins.bin_edges).reduce_range(bottom=bottom_value, top=top_value)
    #background hist
    sp.rebin = split_bins.bin_edges
    bhist = sp.combine_back(feature, reg)
    #data hist
    dhist = sp.make_data_hist(feature, reg, blinded=False)

    #ratio
    flabel = 'Obs. ABCD' if isdata else 'MC ABCD'
    ratio_dict = compare_hists(abcd_hist.make_density_hist(), dhist.make_density_hist(), isdata=1, n=comp_hist_n, 
                                   color = 'red', ndof=5, flabel=flabel, ratio=True, dhist_isdata=1, isabcd=1)

    sp.draw_background(ratio_dict['ax'], feature, reg, sys_label='Stat. + Sys.', errorbar=False)
    #sp.draw_data(ratio_dict['ax'], feature, reg)
    
    handles, labels = ratio_dict['ax'].get_legend_handles_labels()
    order = [0,1,2,3,4,6,5]
    if ismc: order = np.linspace(0,len(handles)-1, len(handles), dtype=int)
    ratio_dict['ax'].legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                                ncol=2)
    
    ratio_dict['ax'].set_ylim(bottom=1e0, top=1e3)
    ratio_dict['fig'].savefig('{}/abcd_mc_ratio_era{}_ismc{}_regSR{}_bottom{}_lognorm_splt_binning.pdf'.format(outdir,era,ismc,nJets,bottom_value))
    #200 GeV zoom in 
    ratio_dict['ax'].set_xlim(left=bottom_value, right=200)
    ratio_dict['rax'].set_xlim(left=bottom_value, right=200)
    ratio_dict['fig'].savefig('{}/abcd_mc_ratio_era{}_ismc{}_regSR{}_bottom{}_lognorm_splt_binning_200GeVMax.pdf'.format(outdir,era,ismc,nJets,bottom_value))
    
    #pull
    pull_dict = compare_hists(abcd_hist.make_density_hist(), dhist.make_density_hist(), isdata=1, n=comp_hist_n, 
                                   color = 'red', ndof=5, flabel=flabel, ratio=False, dhist_isdata=1, isabcd=1)
    sp.draw_background(pull_dict['ax'], feature, reg, sys_label='Stat. + Sys.', errorbar=False)
    handles, labels = pull_dict['ax'].get_legend_handles_labels()
    order = [0,1,2,3,4,6,5]
    if ismc: order = np.linspace(0,len(handles)-1, len(handles), dtype=int)
    pull_dict['ax'].legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                                ncol=2)
    pull_dict['ax'].set_ylim(bottom=1e0, top=1e3)
    pull_dict['fig'].savefig('{}/abcd_mc_pull_era{}_ismc{}_regSR{}_bottom{}_lognorm_splt_binning.pdf'.format(outdir,era,ismc,nJets,bottom_value))
    #200 GeV zoom in 
    pull_dict['ax'].set_xlim(left=bottom_value, right=200)
    pull_dict['rax'].set_xlim(left=bottom_value, right=200)
    pull_dict['fig'].savefig('{}/abcd_mc_pull_era{}_ismc{}_regSR{}_bottom{}_lognorm_splt_binning_200GeVMax.pdf'.format(outdir,era,ismc,nJets,bottom_value))
    sp.rebin = 0

    #save data
    fit_dict['SR{}'.format(nJets)]= ratio_dict
    fit_dict['SR{}'.format(nJets)]['fit_hist'] = abcd_hist

In [ ]:
#transfer factor plot
def make_TF(nJets):
    A = make_fit_hist(bhist, 'CR{}0'.format(nJets)).rebin(split_bins.bin_edges)
    B = make_fit_hist(bhist, 'CR{}3'.format(nJets)).rebin(split_bins.bin_edges)
    C = make_fit_hist(bhist, 'CR{}4'.format(nJets)).rebin(split_bins.bin_edges)
    tf = A.uncertainty_std_dev()/C.uncertainty_std_dev()
    
    tf_nom, tf_std = unp_array_to_nom_std(tf)
    return SysHist(tf_nom, tf_nom*0,tf_nom*0,tf_std, A.bin_edges)
tf_dict = {}
#abcd plots
for nJets in [1,2]:
    reg = 'SR{}'.format(nJets)
    #make abcd hist
    tf_hist = make_TF(nJets)
    tf_hist = tf_hist.reduce_range(bottom=bottom_value, top=top_value)
    fig, ax = plt.subplots()
    A = make_fit_hist(bhist, 'CR{}0'.format(nJets))
    B = make_fit_hist(bhist, 'CR{}3'.format(nJets))
    C = make_fit_hist(bhist, 'CR{}4'.format(nJets))
    #A.calc_ratio(C.nominal).draw(ax)
    tf_hist.draw(ax)
    #C.draw(ax)
    #tf_hist.draw(ax)
    tf_dict[nJets] = tf_hist.to_dict()
    
import pickle as pkl
with open('{}/tf_dict_{}_ismc{}.pkl'.format(outdir, era, ismc), 'wb') as f:
    pkl.dump(tf_dict, f)

In [ ]:
{reg: item['chi2'] for reg, item in fit_dict.items()}

In [ ]:
import inspect

In [ ]:
fit_list = []
for reg, item in fit_dict.items():
    if isdata:
        _plot_dict = sp.make_data_hist(feature, reg).reduce_range(bottom=bottom_value, top=top_value)
    else:
        fig, ax = plt.subplots()
        _plot_dict = sp.draw_background(ax, feature, reg).reduce_range(bottom=bottom_value, top=top_value)
    _fit_dict = {
        "era": era,
        "region": reg,
    'n_{background}': "{:.2f}".format(_plot_dict.uncertainty_std_dev().sum()),
    'n_{ABCD,Data}': "{:.2f}".format(item['fit_hist'].uncertainty_std_dev().sum()),
    ' Data $\chi^2/n_{DOF}$': "{:.2f}".format(item['chi2'])
    }
    if not 'SR' in reg:
        popt_unc = uncertainties.correlated_values(item['popt'], item['pcov'])
        param_names = inspect.getfullargspec(log_norm_np).args[1:]
        treg_dict = {**{n:"{:.2f}".format(u) for u, n in zip(popt_unc, param_names) }}
        _fit_dict = {**_fit_dict, **treg_dict}    
    fit_list.append(_fit_dict)

In [ ]:
with open('{}/fit_stats_data_{}_ismc{}.txt'.format(outdir,era, ismc), 'w') as f:
    latex = pd.DataFrame(fit_list)[['era', 'region', 'n_{background}',  'n_{ABCD,Data}', 'sigma', 'theta', 'mean',
       ' Data $\chi^2/n_{DOF}$']].to_latex(index=False)
    print(latex)
    f.write(latex)

In [ ]:
tdf = pd.DataFrame(fit_list)[['era', 'region', 'n_{background}',  'sigma', 'theta', 'mean', 'n_{ABCD,Data}',
       ' Data $\chi^2/n_{DOF}$']]
tdf

In [ ]:
40665*5778.39/28601

In [ ]:
#format dict for saving
limit_dict = {}
for reg, item in fit_dict.items():
    limit_dict[reg] = item['fit_hist'].to_dict()

In [ ]:
import pickle as pkl
with open('{}/abcd_dict_data_{}_ismc{}.pkl'.format(outdir, era, ismc), 'wb') as f:
    pkl.dump(limit_dict, f)

In [ ]:
fit_dict_skimmed = {}
for k, v in fit_dict.items():
    v = {k2:v2 for k2,v2 in v.items() if not k2 in ['fig', 'ax','rax']}
    fit_dict_skimmed[k] = v

In [ ]:
import pickle as pkl
with open('{}/fit_dict_data_{}_ismc{}.pkl'.format(outdir, era, ismc), 'wb') as f:
    pkl.dump(fit_dict_skimmed, f)

In [ ]:
'{}/abcd_dict_data_{}_ismc{}.pkl'.format(outdir, era, ismc)

In [ ]:
break

In [ ]:
##
## sample from covariance
##

In [ ]:
import copy

In [ ]:
reg = 'CR10'
popt = fit_dict[reg]['popt']
pcov = fit_dict[reg]['pcov']
popt_unc = uncertainties.correlated_values(popt, pcov)

pcov, popt

In [ ]:
def log_norm_unp_safe(*args):
    try:
        return log_norm_unp(*args)
    except:
        return 0
    
def make_hists_region(reg, n_random = 10):
    popt = fit_dict[reg]['popt']
    pcov = fit_dict[reg]['pcov']
    popt_unc = uncertainties.correlated_values(popt, pcov)
    nvars = len(popt)
    # covariance matrix version
    _x = np.array(split_bins.calc_bin_centers())
    _x = np.linspace(120,400, 100)
    
    _y = log_norm_unp(_x, *popt_unc)
    _y_nom, _y_std = unp_array_to_nom_std(_y)
    
    #"square root" of covariance matrix
    w, v = np.linalg.eig(pcov)
    sigma = np.sqrt(w) * v
    #print(np.linalg.cholesky(pcov))
    
    #do extras so we can drop invalid params
    random_popts = np.random.default_rng().multivariate_normal(popt, pcov, n_random)
    
    y_randoms = np.apply_along_axis(lambda p: abs(log_norm_unp_safe(_x, *p)), 1, random_popts)
    y_randoms = [y for y in y_randoms if not type(y)==int]
    #cut down to length
    y_randoms = y_randoms[:n_random]
    #implied mc uncertainties:
    array = np.stack(y_randoms).astype('float')
    mc_unc = array.std(axis=0)
    mc_unc

    return {"y": _y, "y_nom": _y_nom, "y_std": _y_std, "x": _x, "random_popts": random_popts, "y_randoms": np.array(y_randoms).astype('float'), "popt_unc": popt_unc, 'pcov': pcov, 'mc_unc': mc_unc}

In [ ]:
def log_norm_unp_safe(*args):
    try:
        return log_norm_unp(*args)
    except:
        return 0
    
def make_hists_region(reg, n_random = 10):
    popt = fit_dict[reg]['popt']
    pcov = fit_dict[reg]['pcov']
    popt_unc = uncertainties.correlated_values(popt, pcov)
    nvars = len(popt)
    # covariance matrix version
    _x = np.array(split_bins.calc_bin_centers())
    _x = np.linspace(120,400, 100)
    
    _y = log_norm_unp(_x, *popt_unc)
    _y_nom, _y_std = unp_array_to_nom_std(_y)
    
    #"square root" of covariance matrix
    w, v = np.linalg.eig(pcov)
    sigma = np.sqrt(w) * v
    #print(np.linalg.cholesky(pcov))
    
    #do extras so we can drop invalid params
    random_popts = (sigma @ np.random.randn(nvars, int(n_random*1.5))).T + popt
    #random_popts = np.matmul(np.random.rand(n_random, nvars),np.linalg.cholesky(pcov))+popt
    
    y_randoms = list(map(lambda p: abs(log_norm_unp_safe(_x, *p)), random_popts))
    y_randoms = [y for y in y_randoms if not type(y)==int]
    #cut down to length
    y_randoms = y_randoms[:n_random]
    #implied mc uncertainties:
    array = np.stack(y_randoms).astype('float')
    mc_unc = array.std(axis=0)

    return {"y": _y, "y_nom": _y_nom, "y_std": _y_std, "x": _x, "random_popts": random_popts, "y_randoms": np.array(y_randoms).astype('float'), "popt_unc": popt_unc, 'pcov': pcov, 'mc_unc': mc_unc}

In [ ]:
def make_hists_region(reg, n_random = 10):
    popt = fit_dict[reg]['popt']
    pcov = fit_dict[reg]['pcov']
    popt_unc = uncertainties.correlated_values(popt, pcov)
    nvars = len(popt)
    # covariance matrix version
    _x = np.array(split_bins.calc_bin_centers())
    _x = np.linspace(120,400, 100)
    
    _y = log_norm_unp(_x, *popt_unc)
    _y_nom, _y_std = unp_array_to_nom_std(_y)
    
    #"square root" of covariance matrix
    w, v = np.linalg.eig(pcov)
    sigma = np.sqrt(w) * v
    #print(np.linalg.cholesky(pcov))
    
    #do extras so we can drop invalid params
    # https://stats.stackexchange.com/questions/120179/generating-data-with-a-given-sample-covariance-matrix
    random_popts = np.random.default_rng().multivariate_normal(popt, pcov, n_random).T
    means = (random_popts.mean(axis=1)).reshape(-1,1)
    mean_subtracted = random_popts - means
    
    # Make each variable in X orthogonal to one another
    L_inv = np.linalg.cholesky(np.cov(mean_subtracted, bias = True))
    L_inv = np.linalg.inv(L_inv)
    mean_subtracted = np.dot(L_inv, mean_subtracted)
    
    # Rescale X to exactly match Sigma
    L = np.linalg.cholesky(pcov)
    mean_subtracted = np.dot(L, mean_subtracted)
    print("delta covariance matrix: ", pcov, "\n",  np.cov(mean_subtracted))
    #add means back in
    random_popts = mean_subtracted + means
    #rotate it back
    random_popts = random_popts.T

    y_randoms = np.apply_along_axis(lambda p: abs(log_norm_unp_safe(_x, *p)), 1, random_popts)
    y_randoms = [y for y in y_randoms if not sum(y)==0]
    #cut down to length
    y_randoms = y_randoms[:n_random]
    #implied mc uncertainties:
    array = np.stack(y_randoms).astype('float')
    mc_unc = array.std(axis=0)
    mc_unc

    return {"y": _y, "y_nom": _y_nom, "y_std": _y_std, "x": _x, "random_popts": random_popts, "y_randoms": np.array(y_randoms).astype('float'), "popt_unc": popt_unc, 'pcov': pcov, 'mc_unc': mc_unc}


    

In [ ]:
 make_hists_region("CR10")

In [ ]:
import seaborn as sns
def make_cov_plots(cov_dict, region):
    length = len(cov_dict['y_randoms'])
    
    #plot fit information
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    
    #mc fits
    ax.errorbar([0,1], [0,1], color='red', zorder=0, label='MC fit variations')
    for i, y in enumerate(cov_dict['y_randoms']): 
        ax.errorbar(cov_dict['x'],  y/cov_dict['y_nom'], alpha=20/length, color='red', zorder=0)
    
    #covariance fit
    ax.errorbar(cov_dict['x'], cov_dict['y_nom']/cov_dict['y_nom'], yerr=cov_dict['y_std']/cov_dict['y_nom'], zorder=2, label='From covariance')
    
    ax.legend()
    ax.set_xlim(110,400)
    ax.set_ylim(.5, 1.5)
    cms_format_fig(era, ax, "\emph{Preliminary}")
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    ax.set_ylabel('$f(x)_{var.}/f(x)_{nom.}$')
    fig.savefig(f'{outdir}/fit_corr_MC_fits_{era}_{ismc}_{region}.pdf')
    
    #implied error
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    ax.plot(cov_dict['x'], cov_dict['y_std']/cov_dict['y_nom'], label='Std. Dev. from Cov. Matrix')
    ax.plot(cov_dict['x'], cov_dict['mc_unc']/cov_dict['y_nom'], label='Std. Dev. from MC',  color='red')
    ax.set_xlim(110,400)
    ax.set_ylim(0,.3)
    ax.legend()
    ax.set_ylabel('(Per Bin Uncertainty)/Nominal')
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    cms_format_fig(era, ax, "\emph{Preliminary}")
    fig.savefig(f'{outdir}/fit_corr_implied_error_{era}_{ismc}_{region}.pdf')
    
    #correlation plot
    fig, ax = plt.subplots(1,1, figsize=(12,12))
    
    corrcoef = np.corrcoef(cov_dict['y_randoms'].T)
    
    df_corrcoef = pd.DataFrame(corrcoef, index = cov_dict['x'].round(),
                  columns = cov_dict['x'].round())

    sns.heatmap(df_corrcoef, annot=False,
               vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(20, 220, n=200),
        square=True)
    
    #ticks = np.linspace(120, 400, int((400-120)/10+1), dtype='int')
    #ax.set_xticklabels(ticks)
    #ax.set_yticklabels(ticks)
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    ax.set_ylabel('$m_{\ell\ell}$ [GeV]')
    cms_format_fig(era, ax, "\emph{Preliminary}")
    fig.savefig(f'{outdir}/fit_corr_corrplot_{era}_{ismc}_{region}.pdf')
    
    #envelope plot
    
    fig, ax = plt.subplots( figsize=(12,12))
    cov_dict_randoms_sorted = sorted(cov_dict['y_randoms'], key=lambda x: x[0])
    
    #bottom 300
    for i, y in enumerate(cov_dict_randoms_sorted[int(length*(1-.165)): int(length*(1-.155))]): 
        ax.errorbar(cov_dict['x'],  y/cov_dict['y_nom'], alpha=20/length, color='red', zorder=0)
            
    for i, y in enumerate(cov_dict_randoms_sorted[int(length*(.155)): int(length*(.165))]): 
        ax.errorbar(cov_dict['x'],  y/cov_dict['y_nom'], alpha=20/length, color='blue', zorder=0)
        
    
    up = np.array(cov_dict_randoms_sorted[int(length*(1-.165)): int(length*(1-.155))]).mean(axis=0)
    down = np.array(cov_dict_randoms_sorted[int(length*(.155)): int(length*(.165))]).mean(axis=0)
    
    
    ax.errorbar(pdictB['x'],  up/cov_dict['y_nom'], color='blue', zorder=0, label='approx 68\% env up')
    ax.errorbar(pdictB['x'],  down/cov_dict['y_nom'],  color='red', zorder=0, label='approx 68\% env down')
    
    ax.set_ylabel('$f(x)_{var.}/f(x)_{nom.}$')
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    ax.errorbar(cov_dict['x'], cov_dict['y_nom']/cov_dict['y_nom'], yerr=cov_dict['y_std']/cov_dict['y_nom'], zorder=2, label='From covariance')
    cms_format_fig(era, ax, "\emph{Preliminary}")
    ax.set_xlim(110,400)
    ax.set_ylim(.5, 1.5)
    ax.legend()
    fig.savefig(f'{outdir}/fit_corr_envplot_{era}_{ismc}_{region}.pdf')
    
    #fig.savefig('{outdir}/fit_corr_'.format(outdir,era,ismc,nJets,bottom_value))
    

In [ ]:
length = 10000
pdictB = make_hists_region('CR10', n_random=length)

pdictC = make_hists_region('CR13', n_random=length)

pdictD = make_hists_region('CR14', n_random=length)
ABCD_randoms = (pdictB['y_randoms']*pdictC['y_randoms']/pdictD['y_randoms'])
ABCD_unc = ABCD_randoms.std(axis=0)
ABCD = pdictB['y']*pdictC['y']/pdictD['y']
ABCD_nom, ABCD_std = unp_array_to_nom_std(ABCD)   

ABCD_dict = {"y": ABCD, "y_nom": ABCD_nom, "y_std": ABCD_std, "x": pdictD['x'], "y_randoms": ABCD_randoms, "mc_unc":ABCD_unc }

In [ ]:
ABCD_std

In [ ]:
make_cov_plots(ABCD_dict, "SR1")

In [ ]:
make_cov_plots(pdictB, "CR10")

In [ ]:
make_cov_plots(pdictC, "CR13")

In [ ]:
make_cov_plots(pdictD, "CR14")

In [ ]:
length = 1000
pdictB = make_hists_region('CR20', n_random=length)
pdictC = make_hists_region('CR23', n_random=length)
pdictD = make_hists_region('CR24', n_random=length)
ABCD_randoms = (pdictB['y_randoms']*pdictC['y_randoms']/pdictD['y_randoms'])
ABCD_unc = ABCD_randoms.std(axis=0)
ABCD = pdictB['y']*pdictC['y']/pdictD['y']
ABCD_nom, ABCD_std = unp_array_to_nom_std(ABCD)   

ABCD_dict = {"y": ABCD, "y_nom": ABCD_nom, "y_std": ABCD_std, "x": pdictD['x'], "y_randoms": ABCD_randoms, "mc_unc":ABCD_unc }

In [ ]:
make_cov_plots(ABCD_dict)

In [ ]:
make_cov_plots(pdictB)

In [ ]:
# covariance test

In [ ]:
#
pdictB = make_hists_region('CR10', n_random=length)
pdictB.keys()

In [ ]:
# note that the stddev of the random pops is the same as teh stdev from the uncertainty 
pdictB['popt_unc'][0], pdictB['random_popts'][:,0].std()

In [ ]:
#now, lets sort popts and y_randoms by the first value:
def sort_by_index(i):
    return np.stack(sorted(pdictB['random_popts'], key=lambda x:x[i]))

In [ ]:
index = 2
poptsorted = sort_by_index(index)
log_norm_unp

In [ ]:
# def select_envelope(_poptsorted, _index, _envelope=.68):
#     mean, stdev = _poptsorted[:,0].mean(), _poptsorted[:,0].std()
#     up = _poptsorted[]

In [ ]:
mean, stdev = poptsorted[:,index].mean(), poptsorted[:,index].std()
mean, stdev
plt.hist(poptsorted[:,index])

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
x = pdictB['x']
for ls, i in zip([':', "-.", "--"], [.33, .66, 1]):
    sigma =  i
    epsilon = .01
    up_top, up_bottom = mean+stdev*(sigma+epsilon), mean+stdev*(sigma-epsilon) 
    up_pop = poptsorted[(poptsorted[:,index] < up_top) & (poptsorted[:,index] > up_bottom)]
    
    down_top, down_bottom = mean-stdev*(sigma+epsilon), mean-stdev*(sigma-epsilon) 
    down_pop = poptsorted[(poptsorted[:,index] < down_bottom) & (poptsorted[:,index] > down_top)]
    
    x = pdictB['x']
    y_random_up = np.apply_along_axis(lambda p: abs(log_norm_unp_safe(x, *p)), 1, up_pop)
    y_random_up = np.stack([y for y in y_random_up if not type(y)==int])
    
    y_random_down = np.apply_along_axis(lambda p: abs(log_norm_unp_safe(x, *p)), 1, down_pop)
    y_random_down = np.stack([y for y in y_random_down if not type(y)==int])
    
    y_down = y_random_down.mean(axis=0)
    y_up = y_random_up.mean(axis=0)
    y_up.sum(), y_down.sum(), pdictB['y_nom'].sum()
    y_down = y_down/y_down.sum()*pdictB['y_nom'].sum()
    y_up = y_up/y_up.sum()*pdictB['y_nom'].sum()
    
    _ = ax.plot(x, y_down.T/pdictB['y_nom'], label='down {} $\sigma$'.format(i), color='red', ls=ls)
    _ = ax.plot(x, y_up.T/pdictB['y_nom'], label='up {} $\sigma$'.format(i), color='green', ls=ls)
    
    
_ = ax.errorbar(x, pdictB['y_nom']/pdictB['y_nom'], yerr=pdictB['y_std']/pdictB['y_nom'])
ax.legend(ncol=3)

cms_format_fig(era, ax, '\emph{Preliminary}')
ax.set_ylabel('Ratio with Nominal')
ax.set_xlabel('$m_{\ell\ell}$ [GeV]')

In [ ]:
y_randoms_sum = sorted(pdictB['y_randoms'], key=lambda x: sum(x))

foo = lambda x: x[-1]
foo = lambda x: x[-1]-x[0]/47
sums = np.array(list(map(foo, pdictB['y_randoms'])))
mean, std = np.mean(sums), np.std(sums)
mean, std
plt.hist(sums, bins=np.linspace(mean-5*std, mean+5*std, 100))

def get_slice(arr, value, width, foo):
    x = np.array(list(map(foo, arr)))
    print(value*(1-width), value*(1+width))
    xrange = sorted((value*(1-width), value*(1+width)))
    return arr[(x > xrange[0]) &  (x < xrange[1])]


up_slice = get_slice(pdictB['y_randoms'], mean+std, .01, foo)
down_slice = get_slice(pdictB['y_randoms'], mean-std, .01, foo)

y_down = down_slice.mean(axis=0)
y_up = up_slice.mean(axis=0)

In [ ]:
y_up.sum(), y_down.sum(), pdictB['y_nom'].sum()

In [ ]:
_ = plt.plot(y_down.T/pdictB['y_nom'])
_ = plt.plot(y_up.T/pdictB['y_nom'])
_ = plt.plot(pdictB['y_nom']/pdictB['y_nom'])